In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
import numpy as np
import random
import sys
sys.path.append('../tools')

import os

import torch

import pandas as pd
from sklearn.model_selection import train_test_split
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning.callbacks as pl_callbacks
import data_utility, annotation_utility
from models.rns_dataloader import *
from active_learning_utility import get_strategy
from active_learning_data import Data
from active_learning_net import Net
from copy import deepcopy
from models.SwaV import SwaV
from models.SupervisedDownstream import SupervisedDownstream
import warnings

warnings.filterwarnings("ignore", ".*Consider increasing the value of the `num_workers` argument*")
warnings.filterwarnings("ignore", ".*Set a lower value for log_every_n_steps if you want to see logs for the training epoch*")

In [ ]:
random_seed = 42
random.seed(random_seed)
torch.manual_seed(random_seed)
np.random.seed(random_seed)

if torch.cuda.is_available():
    torch.cuda.manual_seed(random_seed)
    # True ensures the algorithm selected by CUFA is deterministic
    torch.backends.cudnn.deterministic = True
    # torch.set_deterministic(True)
    # False ensures CUDA select the same algorithm each time the application is run
    torch.backends.cudnn.benchmark = False

import pytorch_lightning
pytorch_lightning.utilities.seed.seed_everything(seed=random_seed, workers=True)

Global seed set to 42


42

Global seed set to 42


42

In [ ]:
data_dir = "../../../user_data/"
log_folder_root = '../../../user_data/logs/'
ckpt_folder_root = '../../../user_data/checkpoints/'

In [ ]:
strategy_name = 'LeastConfidence'

In [ ]:
nStart = 1
nEnd = 20
nQuery = 2

In [ ]:
args_task = {'n_epoch': 100,
             'transform_train': True,
             'strategy_name': strategy_name,
             'transform': False,
             'loader_tr_args': {'batch_size': 256, 'num_workers': 2, 'collate_fn': collate_fn,
                                'drop_last': True,},
             'loader_te_args': {'batch_size': 256, 'num_workers': 4, 'collate_fn': collate_fn,
                                'drop_last': True,}
             }

In [ ]:
# raw_annotations = pd.read_csv(data_dir + 'full_updated_anns_annotTbl_cleaned.csv')
# ids = list(np.unique(raw_annotations[raw_annotations['descriptions'].notnull()]['HUP_ID']))
# # ids = list(np.unique(raw_annotations['HUP_ID']))
#
# data_import = data_utility.read_files(path=data_dir+'rns_data', path_data=data_dir+'rns_raw_cache', patientIDs=ids,
#                                       verbose=True)  # Import data with annotation

In [ ]:
# data_list = os.listdir(data_dir+'rns_test_cache')
# print(data_list)
data_list = ['HUP047.npy', 'HUP084.npy', 'HUP096.npy', 'HUP109.npy', 'HUP121.npy', 'HUP129.npy', 'HUP131.npy',
             'HUP137.npy', 'HUP147.npy', 'HUP156.npy', 'HUP159.npy', 'HUP182.npy', 'HUP197.npy', 'HUP199.npy',
             'RNS026.npy', 'RNS029.npy']
X_train, y_train, X_test, y_test, index_train, index_test  = get_data(data_list, split=0.8)
# data, label,_,_ = get_data(data_list, split=1)
# train_data, test_data, train_label, test_label = sklearn.model_selection.train_test_split(data, label, test_size=0.8, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

100%|██████████| 16/16 [00:14<00:00,  1.09it/s]

(86197, 249, 36)
(86197,)
(21556, 249, 36)
(21556,)



100%|██████████| 16/16 [00:15<00:00,  1.06it/s]

(86197, 249, 36)
(86197,)
(21556, 249, 36)
(21556,)


In [ ]:
n_pool = len(y_train)
n_test = len(y_test)

NUM_INIT_LB = int(nStart * n_pool / 100)
NUM_QUERY = int(nQuery * n_pool / 100) if nStart != 100 else 0
NUM_ROUND = int((int(nEnd * n_pool / 100) - NUM_INIT_LB) / NUM_QUERY) if nStart != 100 else 0
if NUM_QUERY != 0:
    if (int(nEnd * n_pool / 100) - NUM_INIT_LB) % NUM_QUERY != 0:
        NUM_ROUND += 1

print(NUM_INIT_LB)
print(NUM_QUERY)
print(NUM_ROUND)

861
1723
10
861
1723
10


In [ ]:
dataset = Data(X_train, y_train, X_test, y_test, RNS_Downstream, args_task)

In [ ]:
swav = SwaV().load_from_checkpoint(
    ckpt_folder_root + 'rns_swav_50_12/rns_swav-epoch=82-swav_loss=2.58204.ckpt')
model = SupervisedDownstream(swav.backbone)
# initialize model and save the model state
modelstate = deepcopy(model.state_dict())
device = "cuda" if torch.cuda.is_available() else "cpu"


net = Net(model, args_task, device, ckpt_folder_root = 'rns_active', log_folder_root = 'rns_active')

In [ ]:
strategy = get_strategy(strategy_name, dataset, net, None, args_task)

In [ ]:
# initial round of training, round 0
dataset.initialize_labels(NUM_INIT_LB)
strategy.train()

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\Users\Patrick Xu\Desktop\RNS_Annotation-Pipeline\user_data\checkpoints\rns_active\active_checkpoints_LeastConfidence exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[226, 265, 342, 347, 379, 618, 632, 712, 957, 1003, 1424, 1638, 1841, 2212, 2220, 2225, 2267, 2320, 2449, 2493, 2513, 2544, 2605, 2623, 2685, 2740, 2833, 3050, 3137, 3342, 3502, 3711, 3780, 3804, 4079, 4162, 4272, 4363, 4402, 4472, 4505, 4635, 5012, 5040, 5058, 5069, 5173, 5442, 5453, 5604, 5676, 5866, 5923, 6084, 6459, 6649, 6660, 6662, 6729, 6924, 6956, 7024, 7389, 7478, 7625, 7968, 8131, 8158, 8225, 8296, 8344, 8473, 8854, 8879, 9030, 9249, 9639, 9652, 9724, 9887, 9945, 10085, 10164, 10182, 10262, 10365, 10399, 10463, 10530, 10574, 10593, 10762, 10803, 10815, 11128, 11425, 11597, 11619, 11622, 11751, 11791, 11980, 12042, 12143, 12266, 12450, 12469, 12520, 12582, 12635, 12684, 12691, 12928, 12945, 13022, 13076, 13108, 13132, 13143, 13567, 13694, 13956, 14005, 14117, 14131, 14452, 14475, 14579, 14644, 14813, 15113, 15199, 15267, 15323, 15351, 15371, 15497, 15618, 15697, 15722, 15736, 15844, 16160, 16252, 16275, 16443, 16458, 16467, 16525, 16540, 16754, 16764, 17002, 17147, 17262, 1737

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=100` reached.
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\Users\Patrick Xu\Desktop\RNS_Annotation-Pipeline\user_data\checkpoints\rns_active\active_checkpoints_LeastConfidence exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


[226, 265, 342, 347, 379, 618, 632, 712, 957, 1003, 1424, 1638, 1841, 2212, 2220, 2225, 2267, 2320, 2449, 2493, 2513, 2544, 2605, 2623, 2685, 2740, 2833, 3050, 3137, 3342, 3502, 3711, 3780, 3804, 4079, 4162, 4272, 4363, 4402, 4472, 4505, 4635, 5012, 5040, 5058, 5069, 5173, 5442, 5453, 5604, 5676, 5866, 5923, 6084, 6459, 6649, 6660, 6662, 6729, 6924, 6956, 7024, 7389, 7478, 7625, 7968, 8131, 8158, 8225, 8296, 8344, 8473, 8854, 8879, 9030, 9249, 9639, 9652, 9724, 9887, 9945, 10085, 10164, 10182, 10262, 10365, 10399, 10463, 10530, 10574, 10593, 10762, 10803, 10815, 11128, 11425, 11597, 11619, 11622, 11751, 11791, 11980, 12042, 12143, 12266, 12450, 12469, 12520, 12582, 12635, 12684, 12691, 12928, 12945, 13022, 13076, 13108, 13132, 13143, 13567, 13694, 13956, 14005, 14117, 14131, 14452, 14475, 14579, 14644, 14813, 15113, 15199, 15267, 15323, 15351, 15371, 15497, 15618, 15697, 15722, 15736, 15844, 16160, 16252, 16275, 16443, 16458, 16467, 16525, 16540, 16754, 16764, 17002, 17147, 17262, 1737

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

C:\Users\Patrick Xu\AppData\Local\Programs\Python\Python310\lib\site-packages\pytorch_lightning\trainer\trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
q_idxs = strategy.query(NUM_QUERY)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


data loaded
(85336, 249, 36)
(85336,)


Predicting: 3it [00:00, ?it/s]

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


data loaded
(85336, 249, 36)
(85336,)


Predicting: 1it [00:00, ?it/s]

TypeError: 'NoneType' object is not iterable

In [ ]:
unlabeled_idxs, unlabeled_data = strategy.dataset.get_unlabeled_data()
probs = strategy.predict_prob(unlabeled_data)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


data loaded
(85336, 249, 36)
(85336,)


Predicting: 3it [00:00, ?it/s]

In [51]:
index_train[unlabeled_idxs]

array([(b'HUP047', 1102,  0,   180296), (b'HUP047', 1102,  1,   180296),
       (b'HUP047', 1102,  2,   180296), ...,
       (b'RNS029', 3810, 21, 60804040), (b'RNS029', 3810,  0, 60809715),
       (b'RNS029', 3810,  1, 60809715)],
      dtype=[('patient_index', 'S10'), ('episode_index', '<i4'), ('slice_index', '<i4'), ('start_index', '<i4')])

In [ ]:
uncertainties = probs.max(1)[0]

In [61]:
torch.sort(uncertainties)[1][:2500]

tensor([17853, 36913, 83414,  ...,    49,  4973, 58687])

In [100]:
orders = torch.sort(uncertainties)[1][:5000]
it = index_train[unlabeled_idxs]
orders = orders.numpy()[:,np.newaxis]

In [101]:
it

array([(b'HUP047', 1102,  0,   180296), (b'HUP047', 1102,  1,   180296),
       (b'HUP047', 1102,  2,   180296), ...,
       (b'RNS029', 3810, 21, 60804040), (b'RNS029', 3810,  0, 60809715),
       (b'RNS029', 3810,  1, 60809715)],
      dtype=[('patient_index', 'S10'), ('episode_index', '<i4'), ('slice_index', '<i4'), ('start_index', '<i4')])

In [102]:
x2 = np.sum(orders**2, axis=1) # shape of (m)
y2 = np.sum(orders**2, axis=1) # shape of (n)
xy = np.matmul(orders, orders.T)
x2 = x2.reshape(-1, 1)
dists = np.sqrt(x2 - 2*xy + y2)

In [111]:
np.sort(dists[10])[:100]

array([  0.,   7.,   8.,   9.,  15.,  17.,  53.,  54.,  55.,  64.,  67.,
        71.,  87.,  88.,  95.,  96.,  98., 100., 101., 125., 147., 158.,
       160., 171., 176., 177., 183., 183., 186., 186., 195., 201., 206.,
       226., 227., 233., 234., 239., 240., 244., 249., 269., 280., 301.,
       318., 324., 325., 328., 330., 338., 346., 353., 362., 383., 388.,
       389., 390., 419., 435., 437., 443., 444., 446., 456., 459., 460.,
       464., 466., 474., 494., 495., 509., 523., 527., 546., 554., 555.,
       556., 620., 622., 671., 673., 684., 713., 725., 730., 761., 762.,
       767., 791., 794., 806., 807., 811., 820., 852., 868., 872., 872.,
       873.])

In [106]:
uncertainties[74599:74599]

tensor([0.5011, 0.9974, 0.6502, 0.7940, 0.7307, 0.8481, 0.8499, 0.9987, 0.9709,
        0.7637, 0.9791, 0.9687, 0.9962])

In [99]:
it[61298:61310]

array([(b'HUP159', 1358, 73, 4512020), (b'HUP159', 1358, 74, 4512020),
       (b'HUP159', 1358, 75, 4512020), (b'HUP159', 1358, 76, 4512020),
       (b'HUP159', 1358, 77, 4512020), (b'HUP159', 1358, 78, 4512020),
       (b'HUP159', 1358, 79, 4512020), (b'HUP159', 1358, 80, 4512020),
       (b'HUP159', 1358, 81, 4512020), (b'HUP159', 1358, 82, 4512020),
       (b'HUP159', 1358, 83, 4512020), (b'HUP159', 1359,  0, 4555805)],
      dtype=[('patient_index', 'S10'), ('episode_index', '<i4'), ('slice_index', '<i4'), ('start_index', '<i4')])

In [ ]:
for rd in range(1, NUM_ROUND +1):
    print('round ' + str(rd))
    q_idxs = strategy.query(NUM_QUERY)
    strategy.update(q_idxs)
    strategy.net.round = rd
    strategy.net.net.load_state_dict(modelstate)
    strategy.train()